# Data Warehouse and ETL Pipeline with AWS Redshift and Postgres
## Project: Data Engineering Capstone

This project was completed as part of Udacity's Data Engineering Nanodegree. Data set and prompt provided by [2011–2020 Udacity, Inc.](https://www.udacity.com), used under [CC BY](https://creativecommons.org/licenses/by-nc-nd/3.0/).

#### Project Summary
In this project, we will set up a data warehouse for the US traffic fatality records, specifically data on fatal car crashes that occurred during the years 2016-2018.  

The project follows the following steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

### Step 1: Scope the Project and Gather Data

#### Scope 
A data warehouse will be set up on the AWS Redshift cluster. Raw template traffic and location data will first be staged in AWS S3, and ETL will be performed via Redshift and Postgres to load a star schema of tables for further analysis. Three sets of annual traffic data (2016, 2017, 2018) will be worked on to create three sets (2016, 2017, 2018) of product tables.  
#### Describe and Gather Data 
A set of fatal traffic crash data (csv format) will be gathered from the National Highway Traffic Safety Administration's (NHTSA) Fatality Analysis Reporting System (FARS) [database](ftp://ftp.nhtsa.dot.gov/fars/), which will include the following:

* accident: General crash data.
* drimpair: Impairment of drivers during the crash. 
* nmimpair: Impairment of non-motorists during the crash.
* person: General motorist and non-motorist data.
* vehicle: General vehicle data as well as driver and precrash data.

Also, location data (xlsx format) will be gathered from the U.S. General Services Administration's (GSA) Geographic Locator Codes (GLCs) [database](https://www.gsa.gov/reference/geographic-locator-codes/glcs-for-the-us-and-us-territories):

* glc: US State, City, and County codes and locations referred in the above crash data.

### Step 2: Explore and Assess the Data
#### Explore the Data
We will first explore the data by opening each file (stored in the repository folder called "data") and checking the first few rows. For this quick view, we will just check the 2018 data:

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
#accident
pd.read_csv("data/accident/2018/accident2018.csv").head()

,STATE,ST_CASE,VE_TOTAL,VE_FORMS,PVH_INVL,PEDS,PERNOTMVIT,PERMVIT,PERSONS,COUNTY,CITY,DAY,MONTH,YEAR,DAY_WEEK,HOUR,MINUTE,NHS,RUR_URB,FUNC_SYS,RD_OWNER,ROUTE,TWAY_ID,TWAY_ID2,MILEPT,LATITUDE,LONGITUD,SP_JUR,HARM_EV,MAN_COLL,RELJCT1,RELJCT2,TYP_INT,WRK_ZONE,REL_ROAD,LGT_COND,WEATHER1,WEATHER2,WEATHER,SCH_BUS,RAIL,NOT_HOUR,NOT_MIN,ARR_HOUR,ARR_MIN,HOSP_HR,HOSP_MN,CF1,CF2,CF3,FATALS,DRUNK_DR
0,1,10001,2,1,1,0,0,1,1,121,1870,5,1,2018,6,6,0,1,1,1,1,1,I-20,NaN,1676,33.591331,-86.1319,0,14,0,0,1,1,0,2,4,1,0,1,0,0000000,6,99,6,15,88,88,0,0,0,1,0
1,1,10002,1,1,0,0,0,2,2,127,1780,8,1,2018,2,0,48,1,2,1,1,1,I-22,CALUMET RD,1621,33.809186,-87.2898,0,38,0,1,19,1,0,3,2,2,0,2,0,0000000,0,99,0,59,88,88,20,0,0,2,0
2,1,10003,2,2,0,0,0,2,2,21,0,8,1,2018,2,22,50,1,1,1,1,1,I-65,NaN,2028,32.767736,-86.5640,0,24,0,0,1,1,1,4,2,10,0,10,0,0000000,99,99,23,10,99,99,0,0,0,1,0
3,1,10004,1,1,0,0,0,2,2,3,0,9,1,2018,3,13,2,1,1,1,1,1,I-65,NaN,460,31.027806,-87.6432,0,24,0,0,1,1,0,4,1,10,0,10,0,0000000,13,99,13,14,88,88,0,0,0,1,0
4,1,10005,2,2,0,0,0,2,2,73,330,19,1,2018,6,7,9,1,2,1,1,1,I-459,NaN,17,33.332103,-86.9938,0,12,1,0,1,1,0,1,1,1,0,1,0,0000000,7,99,7,28,88,88,0,0,0,1,0


In [3]:
#drimpair
pd.read_csv("data/drimpair/2018/drimpair2018.csv").head()

,STATE,ST_CASE,VEH_NO,DRIMPAIR
0,1,10001,1,0
1,1,10002,1,99
2,1,10003,1,99
3,1,10003,2,0
4,1,10004,1,99


In [4]:
#nmimpair
pd.read_csv("data/nmimpair/2018/nmimpair2018.csv").head()

,STATE,ST_CASE,VEH_NO,PER_NO,NMIMPAIR
0,1,10006,0,1,0
1,1,10007,0,1,99
2,1,10008,0,1,99
3,1,10010,0,1,99
4,1,10017,0,1,99


In [5]:
#person
pd.read_csv("data/person/2018/person2018.csv").head()

,STATE,ST_CASE,VE_FORMS,VEH_NO,PER_NO,STR_VEH,COUNTY,DAY,MONTH,HOUR,MINUTE,RUR_URB,FUNC_SYS,HARM_EV,MAN_COLL,SCH_BUS,MAKE,MAK_MOD,BODY_TYP,MOD_YEAR,TOW_VEH,SPEC_USE,EMER_USE,ROLLOVER,IMPACT1,FIRE_EXP,AGE,SEX,PER_TYP,INJ_SEV,SEAT_POS,REST_USE,REST_MIS,AIR_BAG,EJECTION,EJ_PATH,EXTRICAT,DRINKING,ALC_DET,ALC_STATUS,ATST_TYP,ALC_RES,DRUGS,DRUG_DET,DSTATUS,HOSPITAL,DOA,DEATH_DA,DEATH_MO,DEATH_YR,DEATH_HR,DEATH_MN,DEATH_TM,LAG_HRS,LAG_MINS,P_SF1,P_SF2,P_SF3,WORK_INJ,HISPANIC,RACE,LOCATION
0,1,10001,1,1,1,0,121,5,1,6,0,1,1,14,0,0,82.0,82881.0,66.0,2018.0,1.0,0.0,0.0,1.0,12.0,0.0,55,1,1,4,11,20,0,20,2,9,0,0,9,2,1,0,0,8,2,0,7,5,1,2018,6,0,600,0,0,0,0,0,1,7,2,0
1,1,10002,1,1,1,0,127,8,1,0,48,2,1,38,0,0,58.0,58037.0,4.0,9999.0,0.0,0.0,0.0,0.0,12.0,1.0,24,1,1,4,11,20,0,1,1,9,0,9,9,0,0,996,9,8,0,0,7,8,1,2018,0,48,48,0,0,0,0,0,0,7,2,0
2,1,10002,1,1,2,0,127,8,1,0,48,2,1,38,0,0,58.0,58037.0,4.0,9999.0,0.0,0.0,0.0,0.0,12.0,1.0,24,1,2,4,13,3,0,1,0,0,9,8,9,0,0,996,8,8,0,0,7,8,1,2018,0,48,48,0,0,0,0,0,0,7,2,0
3,1,10003,2,1,1,0,21,8,1,22,50,1,1,24,0,0,63.0,63402.0,14.0,2008.0,0.0,0.0,0.0,0.0,11.0,0.0,37,2,1,4,11,20,0,1,0,0,9,0,9,2,1,0,9,8,2,0,7,8,1,2018,22,50,2250,0,0,0,0,0,0,7,1,0
4,1,10003,2,2,1,0,21,8,1,22,50,1,1,24,0,0,7.0,7881.0,61.0,2001.0,0.0,0.0,0.0,0.0,1.0,0.0,36,1,1,2,11,20,0,20,0,0,0,0,9,0,0,996,0,8,0,5,0,88,88,8888,88,88,8888,999,99,0,0,0,8,0,0,0


In [6]:
#vehicle
pd.read_csv("data/vehicle/2018/vehicle2018.csv", encoding='windows-1252', low_memory=False).head()

,STATE,ST_CASE,VEH_NO,VE_FORMS,NUMOCCS,DAY,MONTH,HOUR,MINUTE,HARM_EV,MAN_COLL,UNITTYPE,HIT_RUN,REG_STAT,OWNER,MAKE,MODEL,MAK_MOD,BODY_TYP,MOD_YEAR,VIN,VIN_1,VIN_2,VIN_3,VIN_4,VIN_5,VIN_6,VIN_7,VIN_8,VIN_9,VIN_10,VIN_11,VIN_12,TOW_VEH,J_KNIFE,MCARR_I1,MCARR_I2,MCARR_ID,GVWR,V_CONFIG,CARGO_BT,HAZ_INV,HAZ_PLAC,HAZ_ID,HAZ_CNO,HAZ_REL,BUS_USE,SPEC_USE,EMER_USE,TRAV_SP,UNDERIDE,ROLLOVER,ROLINLOC,IMPACT1,DEFORMED,TOWED,M_HARM,VEH_SC1,VEH_SC2,FIRE_EXP,DR_PRES,L_STATE,DR_ZIP,L_STATUS,L_TYPE,CDL_STAT,L_ENDORS,L_COMPL,L_RESTRI,DR_HGT,DR_WGT,PREV_ACC,PREV_SUS1,PREV_SUS2,PREV_SUS3,PREV_DWI,PREV_SPD,PREV_OTH,FIRST_MO,FIRST_YR,LAST_MO,LAST_YR,SPEEDREL,DR_SF1,DR_SF2,DR_SF3,DR_SF4,VTRAFWAY,VNUM_LAN,VSPD_LIM,VALIGN,VPROFILE,VPAVETYP,VSURCOND,VTRAFCON,VTCONT_F,P_CRASH1,P_CRASH2,P_CRASH3,PCRASH4,PCRASH5,ACC_TYPE,TRLR1VIN,TRLR2VIN,TRLR3VIN,DEATHS,DR_DRINK
0,1,10001,1,1,1,5,1,6,0,14,0,1,0,1,2,82,881,82881,66,2018,1FUBGDFG0JLJ,1,F,U,B,G,D,F,G,0,J,L,J,1,1,57,978310,57978310,3,6,1,1,0,0,0,0,0,0,0,60,0,1,1,12,6,2,1,0,0,0,1,1,35211,6,1,6,0,3,0,69,210,0,0,0,0,0,1,0,9,2016,9,2016,0,0,0,0,0,2,2,70,1,1,2,1,0,0,1,11,99,1,4,11,888888888888,777777777777,777777777777,1,0
1,1,10002,1,1,2,8,1,0,48,38,0,1,0,92,0,58,37,58037,4,9999,999999999999,9,9,9,9,9,9,9,9,9,9,9,9,0,0,0,000000000,00000000000,0,0,0,1,0,0,0,0,0,0,0,120,0,0,0,12,6,2,32,0,0,1,1,47,38115,2,1,0,0,2,0,68,140,3,0,0,5,0,0,0,2,2013,9,2015,3,8,37,0,0,2,2,70,1,1,2,2,0,0,1,12,1,1,4,6,777777777777,777777777777,777777777777,2,0
2,1,10003,1,2,1,8,1,22,50,24,0,1,0,1,2,63,402,63402,14,2008,KNDJD7359857,K,N,D,J,D,7,3,5,9,8,5,7,0,0,0,000000000,00000000000,0,0,0,1,0,0,0,0,0,0,0,65,0,0,0,11,6,2,12,0,0,0,1,1,35046,2,1,0,0,2,0,68,185,1,0,2,0,2,0,1,7,2016,9,2017,0,0,0,0,0,2,2,55,1,1,2,2,0,0,1,90,6,1,4,3,777777777777,777777777777,777777777777,1,0
3,1,10003,2,2,1,8,1,22,50,24,0,1,0,1,3,7,881,7881,61,2001,3B6MC366X1M5,3,B,6,M,C,3,6,6,X,1,M,5,0,0,57,1784443,571784443,2,1,96,1,0,0,0,0,0,0,0,70,0,0,0,1,6,2,12,0,0,0,1,1,35180,6,1,0,0,3,0,74,200,1,0,0,0,0,1,3,2,2016,6,2016,0,0,0,0,0,2,2,55,1,1,2,2,0,0,1,61,16,1,1,98,777777777777,777777777777,777777777777,0,0
4,1,10004,1,1,2,9,1,13,2,24,0,1,0,12,3,49,482,49482,34,2004,5TBRN34164S4,5,T,B,R,N,3,4,1,6,4,S,4,0,0,0,000000000,00000000000,0,0,0,1,0,0,0,0,0,0,0,999,0,0,0,11,6,2,24,0,0,0,1,13,30066,1,1,0,0,2,1,74,250,98,0,0,0,0,2,0,12,2013,1,2017,9,58,0,0,0,2,2,70,1,1,2,1,0,0,1,12,7,3,4,2,777777777777,777777777777,777777777777,1,0


In [7]:
#glc
pd.read_excel("data/glc/glc.xlsx").head()

,Territory,State Name,State Code,City Code,City Name,County Code,County Name,Country Code,Old City Name,Date Record Added
0,U,ALABAMA,1,10,ABBEVILLE,67,HENRY,840.0,NaN,NaT
1,U,ALABAMA,1,50,ALBERTVILLE,95,MARSHALL,840.0,NaN,NaT
2,U,ALABAMA,1,60,ALEXANDER CITY,123,TALLAPOOSA,840.0,NaN,NaT
3,U,ALABAMA,1,70,ALICEVILLE,107,PICKENS,840.0,NaN,NaT
4,U,ALABAMA,1,90,ANDALUSIA,39,COVINGTON,840.0,NaN,NaT


Upon inspection and trial code runs, we are concerned with the following five data cleaning issues:

* Duplication: If any, duplicate rows must be removed.
* Nulls: Null/empty values must be removed from the columns of interest.
* Reduction: Unused columns must be dropped in order to reduce our table sizes for faster queries.
* Codification: Since all of the raw traffic data is codified, required details must be entered in according to the [FARS Analytical User's Manual](https://crashstats.nhtsa.dot.gov/Api/Public/ViewPublication/812827) (Exceptions will be the `make` and `model` in the `vehicle` and `cars` tables - due to long lists of values, we will refer our data users to [FARS/CRSS Coding and Validation Manual](https://crashstats.nhtsa.dot.gov/Api/Public/ViewPublication/812828)).
* Column capitalization: Because Redshift is not case sensitive and all column names default to lowercase, this can pose an issue when copying the template data tables. Therefore, all columns must be lowercased.
* City name formatting: First few trial code runs have revealed that many of the city names on the `glc` template table has characters that interfere with proper Redshift loading. To address this, those chracters will be removed from the column. 

We will take the following data cleaning steps:     

#### Cleaning Steps

##### Remove Duplicates

In [8]:
#accident
accident2016df = pd.read_csv("data/accident/2016/accident2016.csv").drop_duplicates()
accident2017df = pd.read_csv("data/accident/2017/accident2017.csv").drop_duplicates()
accident2018df = pd.read_csv("data/accident/2018/accident2018.csv").drop_duplicates()

In [9]:
#drimpair
drimpair2016df = pd.read_csv("data/drimpair/2016/drimpair2016.csv").drop_duplicates()
drimpair2017df = pd.read_csv("data/drimpair/2017/drimpair2017.csv").drop_duplicates()
drimpair2018df = pd.read_csv("data/drimpair/2018/drimpair2018.csv").drop_duplicates()

In [10]:
#nmimpair
nmimpair2016df = pd.read_csv("data/nmimpair/2016/nmimpair2016.csv").drop_duplicates()
nmimpair2017df = pd.read_csv("data/nmimpair/2017/nmimpair2017.csv").drop_duplicates()
nmimpair2018df = pd.read_csv("data/nmimpair/2018/nmimpair2018.csv").drop_duplicates()

In [11]:
#person
person2016df = pd.read_csv("data/person/2016/person2016.csv").drop_duplicates()
person2017df = pd.read_csv("data/person/2017/person2017.csv").drop_duplicates()
person2018df = pd.read_csv("data/person/2018/person2018.csv").drop_duplicates()

In [12]:
#vehicle
vehicle2016df = pd.read_csv("data/vehicle/2016/vehicle2016.csv", encoding='windows-1252', low_memory=False).drop_duplicates()
vehicle2017df = pd.read_csv("data/vehicle/2017/vehicle2017.csv", encoding='windows-1252', low_memory=False).drop_duplicates()
vehicle2018df = pd.read_csv("data/vehicle/2018/vehicle2018.csv", encoding='windows-1252', low_memory=False).drop_duplicates()

In [13]:
#glc
glcdf = pd.read_excel("data/glc/glc.xlsx").drop_duplicates()

##### Remove Nulls and Reduce Tables

In [14]:
def remove_reduce(df, col_list):
    for i in col_list:
        df = df[pd.notnull(df[i])]
    df = df[col_list] 
    return df

In [15]:
#accident
col_list = ['STATE', 'ST_CASE', 'COUNTY', 'CITY', 'DAY', 'MONTH', 'YEAR', 'DAY_WEEK', 'HOUR', 'MINUTE', 
            'LATITUDE', 'LONGITUD', 'FATALS'] 

accident2016df = remove_reduce(accident2016df, col_list)
accident2017df = remove_reduce(accident2017df, col_list)
accident2018df = remove_reduce(accident2018df, col_list)

In [16]:
#drimpair
col_list = ['STATE', 'ST_CASE', 'VEH_NO', 'DRIMPAIR'] 

drimpair2016df = remove_reduce(drimpair2016df, col_list)
drimpair2017df = remove_reduce(drimpair2017df, col_list)
drimpair2018df = remove_reduce(drimpair2018df, col_list)

In [17]:
#nmimpair
col_list = ['STATE', 'ST_CASE', 'VEH_NO', 'PER_NO', 'NMIMPAIR'] 

nmimpair2016df = remove_reduce(nmimpair2016df, col_list)
nmimpair2017df = remove_reduce(nmimpair2017df, col_list)
nmimpair2018df = remove_reduce(nmimpair2018df, col_list)

In [18]:
#person
col_list = ['STATE', 'ST_CASE', 'VEH_NO', 'PER_NO', 'COUNTY', 'DAY', 'MONTH', 'HOUR', 'MINUTE', 'AGE', 'SEX', 
            'PER_TYP', 'INJ_SEV']

person2016df = remove_reduce(person2016df, col_list)
person2017df = remove_reduce(person2017df, col_list)
person2018df = remove_reduce(person2018df, col_list)

In [19]:
#vehicle
col_list = ['STATE', 'ST_CASE', 'VEH_NO', 'DAY', 'MONTH', 'HOUR', 'MINUTE', 'MAKE', 'MODEL', 'MOD_YEAR', 
            'DEFORMED']

vehicle2016df = remove_reduce(vehicle2016df, col_list)
vehicle2017df = remove_reduce(vehicle2017df, col_list)
vehicle2018df = remove_reduce(vehicle2018df, col_list)

In [20]:
#glc
col_list = ['State Name', 'State Code', 'City Code', 'City Name', 'County Code', 'County Name']

glcdf = remove_reduce(glcdf, col_list) 

##### Replace Codes

In [21]:
def replace_codes(df, dic):
    df = df.replace(to_replace=dic, value=None)
    return df

In [22]:
#drimpair
dic = {'DRIMPAIR': {0: 'None/Apparently Normal', 
                    1: "Ill/ Blackout", 
                    2: "Asleep or Fatigued",
                    3: 'Walking with a Cane or Crutches.', 
                    4: 'Paraplegic or in a Wheelchair', 
                    5: 'Impaired Due to Previous Injury', 
                    6: 'Deaf', 
                    7: 'Blind', 
                    8: 'Emotional (Depressed/ Angry/ Disturbed.)', 
                    9: 'Under the Influence of Alcohol/ Drugs or Medication', 
                    10: 'Physical Impairment – No Details', 
                    95: 'No Driver Present/Unknown if Driver Present', 
                    96: 'Other Physical Impairment', 
                    98: 'Not Reported', 
                    99: 'Reported as Unknown if Impaired'}}

drimpair2016df = replace_codes(drimpair2016df, dic)
drimpair2017df = replace_codes(drimpair2017df, dic)
drimpair2018df = replace_codes(drimpair2018df, dic)

In [23]:
#nmimpair
dic = {'NMIMPAIR': {0: 'None/Apparently Normal', 
                    1: 'Ill/ Blackout', 
                    2: 'Asleep or Fatigued',
                    3: 'Walking with a Cane or Crutches.', 
                    4: 'Paraplegic or in a Wheelchair', 
                    5: 'Impaired Due to Previous Injury', 
                    6: 'Deaf', 
                    7: 'Blind', 
                    8: 'Emotional (Depressed/ Angry/ Disturbed.)', 
                    9: 'Under the Influence of Alcohol; Drugs or Medication', 
                    10: 'Physical Impairment – No Details', 
                    95: 'No Driver Present/Unknown if Driver Present', 
                    96: 'Other Physical Impairment', 
                    98: 'Not Reported', 
                    99: 'Unknown'}}

nmimpair2016df = replace_codes(nmimpair2016df, dic)
nmimpair2017df = replace_codes(nmimpair2017df, dic)
nmimpair2018df = replace_codes(nmimpair2018df, dic)

In [24]:
#person
dic = {'SEX': {1: 'Male',
               2: 'Female',
               8: 'Not Reported',
               9: 'Unknown'},
       'PER_TYP': {1: 'Driver of a Motor Vehicle In-Transport', 
                   2: 'Passenger of a Motor Vehicle In-Transport', 
                   3: 'Occupant of a Motor Vehicle Not In-Transport', 
                   4: 'Occupant of a Non-Motor Vehicle Transport Device', 
                   5: 'Pedestrian', 
                   6: 'Bicyclist', 
                   7: 'Other Cyclist', 
                   8: 'Person on Personal Conveyances', 
                   9: 'Unknown Occupant Type in a Motor Vehicle In-Transport', 
                   10: 'Persons In/On Buildings', 
                   19: 'Unknown Type of Non-Motorist'}, 
       'INJ_SEV': {0: 'No Apparent Injury', 
                   1: 'Possible Injury', 
                   2: 'Suspected Minor Injury', 
                   3: 'Suspected Serious Injury', 
                   4: 'Fatal Injury', 
                   5: 'Injured; Severity Unknown', 
                   6: 'Died Prior to Crash', 
                   9: 'Unknown/Not Reported'}}

person2016df = replace_codes(person2016df, dic)
person2017df = replace_codes(person2017df, dic)
person2018df = replace_codes(person2018df, dic)     

In [25]:
#vehicle
dic = {'DEFORMED': {0: 'No Damage', 
                    2: 'Minor Damage', 
                    4: 'Functional Damage',
                    6: 'Disabling Damage',
                    8: 'Not Reported',
                    9: 'Unknown'}} 

vehicle2016df = replace_codes(vehicle2016df, dic)
vehicle2017df = replace_codes(vehicle2017df, dic)
vehicle2018df = replace_codes(vehicle2018df, dic) 

##### Lowercase Columns

In [26]:
def lowercase(df, dic):
    df = df.rename(columns=dic)
    return df

In [27]:
#accident
dic = {'STATE': 'state', 'ST_CASE': 'st_case', 'COUNTY': 'county', 'CITY': 'city', 'DAY': 'day', 'MONTH': 'month', 
       'YEAR': 'year', 'DAY_WEEK': 'day_week', 'HOUR': 'hour', 'MINUTE': 'minute', 'LATITUDE': 'latitude', 
       'LONGITUD': 'longitud', 'FATALS': 'fatals'} 

accident2016df = lowercase(accident2016df, dic)
accident2017df = lowercase(accident2017df, dic)
accident2018df = lowercase(accident2018df, dic)

In [28]:
#drimpair
dic = {'STATE': 'state', 'ST_CASE': 'st_case', 'VEH_NO': 'veh_no', 'DRIMPAIR': 'drimpair'} 

drimpair2016df = lowercase(drimpair2016df, dic)
drimpair2017df = lowercase(drimpair2017df, dic)
drimpair2018df = lowercase(drimpair2018df, dic)

In [29]:
#nmimpair
dic = {'STATE': 'state', 'ST_CASE': 'st_case', 'VEH_NO': 'veh_no', 'PER_NO': 'per_no', 'NMIMPAIR': 'nmimpair'}

nmimpair2016df = lowercase(nmimpair2016df, dic)
nmimpair2017df = lowercase(nmimpair2017df, dic)
nmimpair2018df = lowercase(nmimpair2018df, dic)

In [30]:
#person
dic = {'STATE': 'state', 'ST_CASE': 'st_case', 'VEH_NO': 'veh_no', 'PER_NO': 'per_no', 'COUNTY': 'county', 
       'DAY': 'day', 'MONTH': 'month', 'HOUR': 'hour', 'MINUTE': 'minute', 'AGE': 'age', 'SEX': 'sex', 
       'PER_TYP': 'per_typ', 'INJ_SEV': 'inj_sev'}

person2016df = lowercase(person2016df, dic)
person2017df = lowercase(person2017df, dic)
person2018df = lowercase(person2018df, dic)

In [31]:
#vehicle
dic = {'STATE': 'state', 'ST_CASE': 'st_case', 'VEH_NO': 'veh_no', 'DAY': 'day', 'MONTH': 'month', 'HOUR': 'hour', 
       'MINUTE': 'minute', 'MAKE': 'make', 'MODEL': 'model', 'MOD_YEAR': 'mod_year', 'DEFORMED': 'deformed'}

vehicle2016df = lowercase(vehicle2016df, dic)
vehicle2017df = lowercase(vehicle2017df, dic)
vehicle2018df = lowercase(vehicle2018df, dic)

In [32]:
#glc
dic = {'State Name': 'state_name', 'State Code': 'state_code', 'City Code': 'city_code', 'City Name': 'city_name', 
       'County Code': 'county_code', 'County Name': 'county_name'}

glcdf = lowercase(glcdf, dic)

##### Format City Names

In [33]:
def clean_cities(x):
    for i in ['\n', '"', '"', ',']:
        x = x.replace(i, ' ')
    return x

glcdf['city_name'] = glcdf['city_name'].apply(clean_cities)

We will now save the dataframes into csv files locally and then to S3, under "cleaned_data" directory:

In [2]:
import boto3
import configparser
config = configparser.ConfigParser()
config.read('dwh.cfg')

s3_client = boto3.client('s3', aws_access_key_id=config.get('AWS','aws_access_key_id'), aws_secret_access_key=config.get('AWS','aws_secret_access_key'))

In [39]:
#accident
accident2016df.to_csv ("cleaned_data/accident/2016/accident2016c.csv", index = None, header=True)
accident2017df.to_csv ("cleaned_data/accident/2017/accident2017c.csv", index = None, header=True)
accident2018df.to_csv ("cleaned_data/accident/2018/accident2018c.csv", index = None, header=True)

for i in ['cleaned_data/accident/2016/accident2016c.csv', 'cleaned_data/accident/2017/accident2017c.csv', 'cleaned_data/accident/2018/accident2018c.csv']:
    s3_client.upload_file(f'{i}', config.get('S3','bucket'), f'{i}')

In [40]:
#drimpair
drimpair2016df.to_csv ("cleaned_data/drimpair/2016/drimpair2016c.csv", index = None, header=True)
drimpair2017df.to_csv ("cleaned_data/drimpair/2017/drimpair2017c.csv", index = None, header=True)
drimpair2018df.to_csv ("cleaned_data/drimpair/2018/drimpair2018c.csv", index = None, header=True)

for i in ['cleaned_data/drimpair/2016/drimpair2016c.csv', 'cleaned_data/drimpair/2017/drimpair2017c.csv', 'cleaned_data/drimpair/2018/drimpair2018c.csv']:
    s3_client.upload_file(f'{i}', config.get('S3','bucket'), f'{i}')

In [41]:
#nmimpair
nmimpair2016df.to_csv ("cleaned_data/nmimpair/2016/nmimpair2016c.csv", index = None, header=True)
nmimpair2017df.to_csv ("cleaned_data/nmimpair/2017/nmimpair2017c.csv", index = None, header=True)
nmimpair2018df.to_csv ("cleaned_data/nmimpair/2018/nmimpair2018c.csv", index = None, header=True)

for i in ['cleaned_data/nmimpair/2016/nmimpair2016c.csv', 'cleaned_data/nmimpair/2017/nmimpair2017c.csv', 'cleaned_data/nmimpair/2018/nmimpair2018c.csv']:
    s3_client.upload_file(f'{i}', config.get('S3','bucket'), f'{i}')

In [42]:
#person
person2016df.to_csv ("cleaned_data/person/2016/person2016c.csv", index = None, header=True)
person2017df.to_csv ("cleaned_data/person/2017/person2017c.csv", index = None, header=True)
person2018df.to_csv ("cleaned_data/person/2018/person2018c.csv", index = None, header=True)

for i in ['cleaned_data/person/2016/person2016c.csv', 'cleaned_data/person/2017/person2017c.csv', 'cleaned_data/person/2018/person2018c.csv']:
    s3_client.upload_file(f'{i}', config.get('S3','bucket'), f'{i}')

In [43]:
#vehicle
vehicle2016df.to_csv ("cleaned_data/vehicle/2016/vehicle2016c.csv", index = None, header=True)
vehicle2017df.to_csv ("cleaned_data/vehicle/2017/vehicle2017c.csv", index = None, header=True)
vehicle2018df.to_csv ("cleaned_data/vehicle/2018/vehicle2018c.csv", index = None, header=True)

for i in ['cleaned_data/vehicle/2016/vehicle2016c.csv', 'cleaned_data/vehicle/2017/vehicle2017c.csv', 'cleaned_data/vehicle/2018/vehicle2018c.csv']:
    s3_client.upload_file(f'{i}', config.get('S3','bucket'), f'{i}')

In [44]:
#glc
glcdf.to_csv ("cleaned_data/glc/glcc.csv", index = None, header=True)

for i in ['cleaned_data/glc/glcc.csv']:
    s3_client.upload_file(f'{i}', config.get('S3','bucket'), f'{i}')

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model

For the most optimal organization, joins, and queries, we will perform dimensional data modeling to create the following star schema:

**Fact Table**

1. **crashes**

    - case_id, date_id, loc_id, car_id1, car_id2, car_id3, car_id4, nm_id1, nm_id2, nm_id3, nm_id4, fatalities

**Dimension Tables**

2. **dates**
    - date_id, case_id, year, month, day, weekday, hour, minute

3. **location**
    - loc_id, case_id, state, city, county, latitude, longitude
    
4. **cars**
    - car_id, case_id, make, model, mod_year, damage, dr_age, dr_sex, dr_injury, dr_impair
    
5. **non_motorists**
    - nm_id, case_id, nm_type, nm_age, nm_sex, nm_injury, nm_impair


#### 3.2 Mapping Out Data Pipelines

We will take the following steps to create our data model:

1. Create empty tables in the Redshift cluster.
2. Load the csv file data into template redshift tables (`accident`, `drimpair`, `nmimpair`, `person`, `vehicle`, `glc`).
3. Query and join the template tables to reorganize and load the data into the fact and dimension tables.  

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model

##### Connect API to Redshift 
We begin by connecting our Jupyter Notebook to Redshift cluster and launching session. To test, user must launch a Redshift cluster on AWS with API and S3 access enabled, and enter the required credentials in the dwh.cfg file:

In [3]:
import psycopg2
conn = psycopg2.connect(f"host={config.get('REDSHIFT','host')} dbname={config.get('REDSHIFT','dbname')} user={config.get('REDSHIFT','user')} password={config.get('REDSHIFT','password')} port={config.get('REDSHIFT','port')}")
conn.set_session(autocommit=True)
cur = conn.cursor()

##### Create Tables
Now we create the tables on Redshift. Table dropping query is executed first each time to allow data reset.

In [46]:
#accident
for i in ["2016", "2017", "2018"]:
    accident_drop = f"DROP table IF EXISTS accident{i}"
    accident_create = f"""
    CREATE TABLE IF NOT EXISTS accident{i} 
    (state int, st_case int, county int, city int, day int, month int, year int, day_week int, hour int, minute int, 
    latitude decimal, longitud decimal, fatals int)
    """
    cur.execute(accident_drop)
    cur.execute(accident_create)

In [47]:
#drimpair
for i in ["2016", "2017", "2018"]:
    drimpair_drop = f"DROP table IF EXISTS drimpair{i}"
    drimpair_create = f"""
    CREATE TABLE IF NOT EXISTS drimpair{i} 
    (state int, st_case int, veh_no int, drimpair text)
    """
    cur.execute(drimpair_drop)
    cur.execute(drimpair_create)

In [48]:
#nmimpair
for i in ["2016", "2017", "2018"]:
    nmimpair_drop = f"DROP table IF EXISTS nmimpair{i}"
    nmimpair_create = f"""
    CREATE TABLE IF NOT EXISTS nmimpair{i} 
    (state int, st_case int, veh_no int, per_no int, nmimpair text)
    """
    cur.execute(nmimpair_drop)
    cur.execute(nmimpair_create)

In [49]:
#person
for i in ["2016", "2017", "2018"]:
    person_drop = f"DROP table IF EXISTS person{i}"
    person_create = f"""
    CREATE TABLE IF NOT EXISTS person{i}
    (state int, st_case int, veh_no int, per_no int, county int, day int, month int, hour int, minute int, age int, 
    sex text, per_typ text, inj_sev text)
    """
    cur.execute(person_drop)
    cur.execute(person_create)

In [50]:
#vehicle
for i in ["2016", "2017", "2018"]:
    vehicle_drop = f"DROP table IF EXISTS vehicle{i}"
    vehicle_create = f"""
    CREATE TABLE IF NOT EXISTS vehicle{i}
    (state int, st_case int, veh_no int, day int, month int, hour int, minute int, make int, model int, mod_year int, 
    deformed text)
    """
    cur.execute(vehicle_drop)
    cur.execute(vehicle_create)

In [51]:
#glc
glc_drop = f"DROP table IF EXISTS glc"
glc_create = f"""
CREATE TABLE IF NOT EXISTS glc 
(state_name text, state_code int, city_code int, city_name text, county_code int, county_name text)
"""
cur.execute(glc_drop)
cur.execute(glc_create)

In [52]:
#crashes
for i in ["2016", "2017", "2018"]:
    crashes_drop = f"DROP table IF EXISTS crashes{i}"
    crashes_create = f"""
    CREATE TABLE IF NOT EXISTS crashes{i}
    (case_id int, date_id int, loc_id int, car_id1 numeric, car_id2 numeric, car_id3 numeric, car_id4 numeric, 
    nm_id1 numeric, nm_id2 numeric, nm_id3 numeric, nm_id4 numeric, fatalities int)
    """
    cur.execute(crashes_drop)
    cur.execute(crashes_create)

In [53]:
#dates
for i in ["2016", "2017", "2018"]:
    dates_drop = f"DROP table IF EXISTS dates{i}"
    dates_create = f"""
    CREATE TABLE IF NOT EXISTS dates{i}
    (date_id int identity(0,1), case_id int, year int, month int, day int, weekday int, hour int, minute int)
    """
    cur.execute(dates_drop)
    cur.execute(dates_create)

In [54]:
#location
for i in ["2016", "2017", "2018"]:
    location_drop = f"DROP table IF EXISTS location{i}"
    location_create = f"""
    CREATE TABLE IF NOT EXISTS location{i}
    (loc_id int identity(0,1), case_id int, state text, city text, county text, latitude numeric, longitude numeric)
    """
    cur.execute(location_drop)
    cur.execute(location_create)

In [55]:
#cars
for i in ["2016", "2017", "2018"]:
    cars_drop = f"DROP table IF EXISTS cars{i}"
    cars_create = f"""
    CREATE TABLE IF NOT EXISTS cars{i}
    (car_id int identity(0,1), case_id int, make int, model int, mod_year int, damage text, dr_age int, 
    dr_sex text, dr_injury text, dr_impair text)
    """
    cur.execute(cars_drop)
    cur.execute(cars_create)

In [56]:
#non_motorists
for i in ["2016", "2017", "2018"]:
    non_motorists_drop = f"DROP table IF EXISTS non_motorists{i}"
    non_motorists_create = f"""
    CREATE TABLE IF NOT EXISTS non_motorists{i}
    (nm_id int identity(0,1), case_id int, nm_type text, nm_age int, nm_sex text, nm_injury text, 
    nm_impair text)
    """
    cur.execute(non_motorists_drop)
    cur.execute(non_motorists_create)

##### Load Template Tables
Now we load the template tables (`accident`, `drimpair`, `nmimpair`, `person`, `vehicle`, `glc`):

In [57]:
for i in ['accident2016', 'drimpair2016', 'nmimpair2016', 'person2016', 'vehicle2016',
          'accident2017', 'drimpair2017', 'nmimpair2017', 'person2017', 'vehicle2017',
          'accident2018', 'drimpair2018', 'nmimpair2018', 'person2018', 'vehicle2018']:
    load_traffic_query = f"""
    COPY {i}
    FROM 's3://{config.get('S3','bucket')}/cleaned_data/{i[:-4]}/{i[-4:]}'
    ACCESS_KEY_ID '{config.get('AWS','aws_access_key_id')}'
    SECRET_ACCESS_KEY '{config.get('AWS','aws_secret_access_key')}'
    IGNOREHEADER 1
    DELIMITER ','
    """
    cur.execute(load_traffic_query)

load_glc_query = f"""
COPY glc
FROM 's3://{config.get('S3','bucket')}/cleaned_data/glc'
ACCESS_KEY_ID '{config.get('AWS','aws_access_key_id')}'
SECRET_ACCESS_KEY '{config.get('AWS','aws_secret_access_key')}'
IGNOREHEADER 1
DELIMITER ','
"""
cur.execute(load_glc_query)

##### Transform Data and Load Fact and Dimension Tables

From our template tables, we will transform the data via query and load it into our fact and dimension tables: 

In [58]:
#dates
for i in ["2016", "2017", "2018"]:
    dates_insert = f"""
    INSERT INTO dates{i} (case_id, year, month, day, weekday, hour, minute)
    SELECT st_case,
           year,
           month,
           day,
           day_week,
           hour,
           minute
    FROM accident{i};
    """
    cur.execute(dates_insert)

In [59]:
#location
for i in ["2016", "2017", "2018"]:
    location_insert = f"""
    INSERT INTO location{i} (case_id, state, city, county, latitude, longitude)
    SELECT a.st_case,
           g.state_name,
           g.city_name,
           g.county_name, 
           a.latitude,
           a.longitud
    FROM accident{i} a
    JOIN glc g
    ON a.state = g.state_code AND  
       a.county = g.county_code AND 
       a.city = g.city_code
    """
    cur.execute(location_insert)

In [60]:
#cars (Note: Drivers correspond to per_no = 1 in the person template table)
for i in ["2016", "2017", "2018"]:
    cars_insert = f"""
    INSERT INTO cars{i} (case_id, make, model, mod_year, damage, dr_age, dr_sex, dr_injury, dr_impair)
    SELECT v.st_case,
           v.make,
           v.model,
           v.mod_year, 
           v.deformed,
           p.age,
           p.sex,
           p.inj_sev,
           d.drimpair
    FROM vehicle{i} v
    JOIN (SELECT *
          FROM person{i}
          WHERE per_no = 1) p
    ON v.state = p.state AND  
       v.st_case = p.st_case AND 
       v.veh_no = p.veh_no
    JOIN drimpair{i} d
    ON v.state = d.state AND  
       v.st_case = d.st_case AND 
       v.veh_no = d.veh_no
    """
    cur.execute(cars_insert)

In [61]:
#non_motorists (Note: Non_motorists correspond to veh_no = 0 in the person template table)
for i in ["2016", "2017", "2018"]:
    non_motorists_insert = f"""
    INSERT INTO non_motorists{i} (case_id, nm_type, nm_age, nm_sex, nm_injury, nm_impair)
    SELECT p.st_case,
           p.per_typ,
           p.age,
           p.sex, 
           p.inj_sev,
           n.nmimpair
    FROM (SELECT *
          FROM person{i}
          WHERE veh_no = 0) p
    JOIN nmimpair{i} n
    ON p.state = n.state AND  
       p.st_case = n.st_case AND 
       p.veh_no = n.veh_no
    """
    cur.execute(non_motorists_insert)

For the `crashes` fact table, all the dimension tables must be joined to the `accident` table, but the tables `cars` and `non_motorists` must be pivoted first to move `car_id` and `nm_id` rows to columns. Unfortunately, Redshift does not have any table function for pivot, but [Postgres does](https://www.postgresql.org/docs/9.2/tablefunc.html). Therefore, for this final table, we will utilize Postgres - we will create a Postgres database on AWS, load required segments of the dimension tables, alter and join them, and load the product back into Redshift to finish creating the `crashes` fact table. 

We first begin by extracting required segments:

In [4]:
#dates_segment
dsq = """ 
SELECT case_id,
       date_id
FROM dates{}
"""
dates_segment2016df = pd.read_sql_query(dsq.format('2016'), conn)
dates_segment2017df = pd.read_sql_query(dsq.format('2017'), conn)
dates_segment2018df = pd.read_sql_query(dsq.format('2018'), conn)

In [5]:
#location_segment
lsq = """ 
SELECT case_id,
       loc_id
FROM location{}
"""
location_segment2016df = pd.read_sql_query(lsq.format('2016'), conn)
location_segment2017df = pd.read_sql_query(lsq.format('2017'), conn)
location_segment2018df = pd.read_sql_query(lsq.format('2018'), conn)

In [6]:
#cars_segment
csq = """ 
SELECT case_id,
       make,
       car_id
FROM cars{}
"""
cars_segment2016df = pd.read_sql_query(csq.format('2016'), conn)
cars_segment2017df = pd.read_sql_query(csq.format('2017'), conn)
cars_segment2018df = pd.read_sql_query(csq.format('2018'), conn)

In [7]:
#non_motorists_segment
nmsq = """ 
SELECT case_id,
       nm_type,
       nm_id
FROM non_motorists{}
""" 
non_motorists_segment2016df = pd.read_sql_query(nmsq.format('2016'), conn) 
non_motorists_segment2017df = pd.read_sql_query(nmsq.format('2017'), conn) 
non_motorists_segment2018df = pd.read_sql_query(nmsq.format('2018'), conn)

In [8]:
#accident_segment
asq = """ 
SELECT st_case,
       fatals
FROM accident{}
""" 
accident_segment2016df = pd.read_sql_query(asq.format('2016'), conn) 
accident_segment2017df = pd.read_sql_query(asq.format('2017'), conn) 
accident_segment2018df = pd.read_sql_query(asq.format('2018'), conn)

For now, we will close the Redshift connection and connect to a Postgres database (To test, user must create his own Postgres instance and enter the credentials in the dwh.cfg file). We will then load the extracted segment dataframes into the the Postgres database, utilizing Pandas' [to_sql method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html) (latest version of Pandas is required to utilize `method="multi"` fast load parameter).  

In [9]:
#close Redshift connection
conn.close()

In [10]:
#connect to Postgres
import psycopg2
conn = psycopg2.connect(f"host={config.get('POSTGRES','host')} dbname={config.get('POSTGRES','dbname')} user={config.get('POSTGRES','user')} password={config.get('POSTGRES','password')} port={config.get('POSTGRES','port')}")
conn.set_session(autocommit=True, readonly=False)
cur = conn.cursor()

In [11]:
#load segments to Postgres
from sqlalchemy import create_engine
engine = create_engine(f"postgresql://{config.get('POSTGRES','user')}:{config.get('POSTGRES','password')}@{config.get('POSTGRES','host')}:{config.get('POSTGRES','port')}/{config.get('POSTGRES','dbname')}")

dates_segment2016df.to_sql('dates_segment2016', engine, method="multi", index=False)
dates_segment2017df.to_sql('dates_segment2017', engine, method="multi", index=False)
dates_segment2018df.to_sql('dates_segment2018', engine, method="multi", index=False)

location_segment2016df.to_sql('location_segment2016', engine, method="multi", index=False)
location_segment2017df.to_sql('location_segment2017', engine, method="multi", index=False)
location_segment2018df.to_sql('location_segment2018', engine, method="multi", index=False)

cars_segment2016df.to_sql('cars_segment2016', engine, method="multi", index=False)
cars_segment2017df.to_sql('cars_segment2017', engine, method="multi", index=False)
cars_segment2018df.to_sql('cars_segment2018', engine, method="multi", index=False)

non_motorists_segment2016df.to_sql('non_motorists_segment2016', engine, method="multi", index=False)
non_motorists_segment2017df.to_sql('non_motorists_segment2017', engine, method="multi", index=False)
non_motorists_segment2018df.to_sql('non_motorists_segment2018', engine, method="multi", index=False)

accident_segment2016df.to_sql('accident_segment2016', engine, method="multi", index=False)
accident_segment2017df.to_sql('accident_segment2017', engine, method="multi", index=False)
accident_segment2018df.to_sql('accident_segment2018', engine, method="multi", index=False)

We now perform our pivot process on the `cars_segment` and `non_motorists_segment`:

##### Disclaimer: Because pivoting creates null values in car_id and nm_id columns 2,3,4, float column type must be used to accomodate them. For clairty and practicality of future joins, we will use floats for all `car_id`s and `nm_id`s; we will also return to `cars` and `non_motorists` tables later to fix the columns.  

In [12]:
#install tablefunc module to access crosstab function
create_ct = "CREATE EXTENSION tablefunc;"
cur.execute(create_ct)

In [13]:
#convert car_id and nm_id to floats
for i in ['cars_segment2016', 'cars_segment2017', 'cars_segment2018']:
    alter_column = f"""ALTER TABLE {i} ALTER COLUMN car_id TYPE numeric"""
    cur.execute(alter_column)

for i in ['non_motorists_segment2016', 'non_motorists_segment2017', 'non_motorists_segment2018']:
    alter_column = f"""ALTER TABLE {i} ALTER COLUMN nm_id TYPE numeric"""
    cur.execute(alter_column)

In [14]:
for i in ['cars_segment2016', 'cars_segment2017', 'cars_segment2018']:
    cs_pivot_drop = f"""DROP TABLE IF EXISTS {i}ct"""
    cs_pivot_create = f"""CREATE TABLE IF NOT EXISTS {i}ct(case_id int, car_id1 numeric, car_id2 numeric, car_id3 numeric, 
                                                           car_id4 numeric)"""  
    cs_pivot_insert = f"""INSERT INTO {i}ct
                          SELECT * 
                          FROM crosstab('SELECT * 
                                         FROM {i}
                                         ORDER BY 1,2') 
                          AS {i}(case_id bigint, car_id1 numeric, car_id2 numeric, car_id3 numeric, car_id4 numeric);"""
    cur.execute(cs_pivot_drop)
    cur.execute(cs_pivot_create)
    cur.execute(cs_pivot_insert)

for i in ['non_motorists_segment2016', 'non_motorists_segment2017', 'non_motorists_segment2018']:
    nms_pivot_drop = f"""DROP TABLE IF EXISTS {i}ct"""
    nms_pivot_create = f"""CREATE TABLE IF NOT EXISTS {i}ct(case_id int, nm_id1 numeric, nm_id2 numeric, nm_id3 numeric, 
                                                            nm_id4 numeric)""" 
    nms_pivot_insert = f"""INSERT INTO {i}ct
                           SELECT * 
                           FROM crosstab('SELECT * 
                                          FROM {i}
                                          ORDER BY 1,2') 
                           AS {i}(case_id bigint, nm_id1 numeric, nm_id2 numeric, bignm_id3 numeric, nm_id4 numeric);"""
    cur.execute(nms_pivot_drop)
    cur.execute(nms_pivot_create)
    cur.execute(nms_pivot_insert)

We then join all the segments and load the crosstab products into the `crashes` table:

In [15]:
#create crashes table in Postgres
for i in ["2016", "2017", "2018"]:
    crashes_drop = f"DROP table IF EXISTS crashes{i}"
    crashes_create = f"""
    CREATE TABLE IF NOT EXISTS crashes{i}
    (case_id int, date_id int, loc_id int, car_id1 numeric, car_id2 numeric, car_id3 numeric, car_id4 numeric, 
    nm_id1 numeric, nm_id2 numeric, nm_id3 numeric, nm_id4 numeric, fatalities int)
    """
    cur.execute(crashes_drop)
    cur.execute(crashes_create)

In [16]:
#join and load
for i in ["2016", "2017", "2018"]:
    crashes_insert = f"""
    INSERT INTO crashes{i}
    SELECT a.st_case,
           d.date_id,
           l.loc_id,
           c.car_id1, 
           c.car_id2,
           c.car_id3,
           c.car_id4,
           nm.nm_id1,
           nm.nm_id2,
           nm.nm_id3,
           nm.nm_id4,
           a.fatals
    FROM accident_segment{i} a
    JOIN dates_segment{i} d
    ON a.st_case = d.case_id
    JOIN location_segment{i} l
    ON a.st_case = l.case_id
    JOIN cars_segment{i}ct c
    ON a.st_case = c.case_id
    JOIN non_motorists_segment{i}ct nm
    ON a.st_case = nm.case_id
    """
    cur.execute(crashes_insert)

Finally, the `crashes` table will be retrieved from Postgres to the Jupyter Notebook API and saved, then loaded to Redshift:

In [17]:
#extract to API
cq = """ 
SELECT *
FROM crashes{}
"""
crashes2016df = pd.read_sql_query(cq.format('2016'), conn)
crashes2017df = pd.read_sql_query(cq.format('2017'), conn)
crashes2018df = pd.read_sql_query(cq.format('2018'), conn)

In [18]:
#save locally and to S3
crashes2016df.to_csv ("cleaned_data/crashes/2016/crashes2016c.csv", index = None, header=True)
crashes2017df.to_csv ("cleaned_data/crashes/2017/crashes2017c.csv", index = None, header=True)
crashes2018df.to_csv ("cleaned_data/crashes/2018/crashes2018c.csv", index = None, header=True)

for i in ['cleaned_data/crashes/2016/crashes2016c.csv', 'cleaned_data/crashes/2017/crashes2017c.csv', 'cleaned_data/crashes/2018/crashes2018c.csv']:
    s3_client.upload_file(f'{i}', config.get('S3','bucket'), f'{i}')

In [19]:
#disconnect from Postgres and reconnect to Redshift

conn.close()

conn = psycopg2.connect(f"host={config.get('REDSHIFT','host')} dbname={config.get('REDSHIFT','dbname')} user={config.get('REDSHIFT','user')} password={config.get('REDSHIFT','password')} port={config.get('REDSHIFT','port')}")
conn.set_session(autocommit=True)
cur = conn.cursor()

In [20]:
#create crashes table
for i in ["2016", "2017", "2018"]:
    crashes_drop = f"DROP table IF EXISTS crashes{i}"
    crashes_create = f"""
    CREATE TABLE IF NOT EXISTS crashes{i}
    (case_id int, date_id int, loc_id int, car_id1 numeric, car_id2 numeric, car_id3 numeric, car_id4 numeric, 
    nm_id1 numeric, nm_id2 numeric, nm_id3 numeric, nm_id4 numeric, fatalities int)
    """
    cur.execute(crashes_drop)
    cur.execute(crashes_create)

In [21]:
#load crashes table
for i in ['crashes2016', 'crashes2017', 'crashes2018']:
    load_crashes_query = f"""
    COPY {i}
    FROM 's3://{config.get('S3','bucket')}/cleaned_data/{i[:-4]}/{i[-4:]}'
    ACCESS_KEY_ID '{config.get('AWS','aws_access_key_id')}'
    SECRET_ACCESS_KEY '{config.get('AWS','aws_secret_access_key')}'
    IGNOREHEADER 1
    DELIMITER ','
    """
    cur.execute(load_crashes_query)

Finally, we finish by fixing our `car_id` and `nm_id` columns in `cars` and `non_motorists` tables, respectively:

In [22]:
#cars
for i in ['2016', '2017','2018']:
    fix_cars = f"""
                    BEGIN TRANSACTION;

                    ALTER TABLE cars{i} RENAME TO cars{i}old;
                    CREATE TABLE cars{i} (car_id numeric, case_id int, make int, model int, mod_year int, damage text, 
                                          dr_age int, dr_sex text, dr_injury text, dr_impair text);
                    INSERT INTO cars{i}
                    SELECT CAST(car_id as numeric),
                           case_id,
                           make,
                           model,
                           mod_year,
                           damage,
                           dr_age,
                           dr_sex,
                           dr_injury,
                           dr_impair
                    FROM cars{i}OLD;
                    DROP TABLE cars{i}OLD;

                    END TRANSACTION;
                """
    cur.execute(fix_cars)

In [23]:
#non_motorists
for i in ['2016', '2017','2018']:
    fix_nm = f"""
                    BEGIN TRANSACTION;

                    ALTER TABLE non_motorists{i} RENAME TO non_motorists{i}old;
                    CREATE TABLE non_motorists{i} (nm_id numeric, case_id int, nm_type text, nm_age int, nm_sex text, 
                                                   nm_injury text, nm_impair text);
                    INSERT INTO non_motorists{i}
                    SELECT CAST(nm_id as numeric),
                           case_id,
                           nm_type, 
                           nm_age, 
                           nm_sex, 
                           nm_injury, 
                           nm_impair
                    FROM non_motorists{i}OLD;
                    DROP TABLE non_motorists{i}OLD;

                    END TRANSACTION;
                """
    cur.execute(fix_nm)

#### 4.2 Data Quality Checks

Now that all the tables have been loaded on to Redshift, we will execute data quality checks. We will perform the following checks:

* Count: Confirm that correct numbers of rows have been inserted. 
* Datatype: Confirm that each column does have the correct datatype.

##### Count Checks

In [24]:
#dates
for i in ['2016', '2017', '2018']:
    cc = f"""SELECT COUNT(a.st_case) AS expected_rows, 
                    COUNT(d.case_id) AS inserted_rows,
                    CASE WHEN expected_rows = inserted_rows THEN 'PASS'
                         WHEN expected_rows != inserted_rows THEN 'FAIL'
                         END AS count_check_dates{i}
             FROM accident{i} a
             JOIN dates{i} d
             ON a.st_case = d.case_id
      """
    print(pd.read_sql_query(cc, conn))

   expected_rows  inserted_rows count_check_dates2016
0          34748          34748                  PASS
   expected_rows  inserted_rows count_check_dates2017
0          34560          34560                  PASS
   expected_rows  inserted_rows count_check_dates2018
0          33654          33654                  PASS


In [25]:
#location
for i in ['2016', '2017', '2018']:
    cc = f"""SELECT COUNT(a.st_case) AS expected_rows, 
                    COUNT(l.case_id) AS inserted_rows, 
                    CASE WHEN expected_rows = inserted_rows THEN 'PASS'
                         WHEN expected_rows != inserted_rows THEN 'FAIL'
                         END AS count_check_location{i}
            FROM accident{i} a
            JOIN location{i} l
            ON a.st_case = l.case_id
      """
    print(pd.read_sql_query(cc, conn))

   expected_rows  inserted_rows count_check_location2016
0          17118          17118                     PASS
   expected_rows  inserted_rows count_check_location2017
0          17324          17324                     PASS
   expected_rows  inserted_rows count_check_location2018
0          16507          16507                     PASS


In [26]:
#cars
for i in ['2016', '2017', '2018']:
    cc = f"""SELECT COUNT(v.st_case) AS expected_rows, 
                    COUNT(c.case_id) AS inserted_rows, 
                    CASE WHEN expected_rows = inserted_rows THEN 'PASS'
                         WHEN expected_rows != inserted_rows THEN 'FAIL'
                         END AS count_check_cars{i}
             FROM vehicle{i} v
             JOIN cars{i} c
             ON v.st_case = c.case_id
      """
    print(pd.read_sql_query(cc, conn))

   expected_rows  inserted_rows count_check_cars2016
0         102652         102652                 PASS
   expected_rows  inserted_rows count_check_cars2017
0         102995         102995                 PASS
   expected_rows  inserted_rows count_check_cars2018
0          99862          99862                 PASS


In [27]:
#non_motorists 
for i in ['2016', '2017', '2018']:
    cc = f"""SELECT COUNT(p.st_case) AS expected_rows, 
                    COUNT(nm.case_id) AS inserted_rows, 
                    CASE WHEN expected_rows = inserted_rows THEN 'PASS'
                         WHEN expected_rows != inserted_rows THEN 'FAIL'
                         END AS count_check_non_motorists{i}
             FROM (SELECT *
                   FROM person{i}
                   WHERE veh_no = 0) p
             JOIN non_motorists{i} nm
             ON p.st_case = nm.case_id
      """ 
    print(pd.read_sql_query(cc, conn))

   expected_rows  inserted_rows count_check_non_motorists2016
0          17733          17733                          PASS
   expected_rows  inserted_rows count_check_non_motorists2017
0          27799          27799                          PASS
   expected_rows  inserted_rows count_check_non_motorists2018
0          16523          16523                          PASS


In [28]:
#crashes
for i in ['2016', '2017', '2018']:
    cc = f"""SELECT COUNT(a.st_case) AS expected_rows, 
                    COUNT(cr.case_id) AS inserted_rows, 
                    CASE WHEN expected_rows = inserted_rows THEN 'PASS'
                         WHEN expected_rows != inserted_rows THEN 'FAIL'
                         END AS count_check_crashes{i}
             FROM accident{i} a
             JOIN crashes{i} cr
             ON a.st_case = cr.case_id
      """
    print(pd.read_sql_query(cc, conn))

   expected_rows  inserted_rows count_check_crashes2016
0           5000           5000                    PASS
   expected_rows  inserted_rows count_check_crashes2017
0           4932           4932                    PASS
   expected_rows  inserted_rows count_check_crashes2018
0           5065           5065                    PASS


##### Datatype Checks

In [29]:
#dates
for i in ['2016', '2017', '2018']:
    dc = f"""SELECT "column",
                    CASE WHEN "column" = 'date_id' THEN 'integer' 
                         WHEN "column" = 'case_id' THEN 'integer' 
                         WHEN "column" = 'year' THEN 'integer' 
                         WHEN "column" = 'month' THEN 'integer'
                         WHEN "column" = 'day' THEN 'integer'
                         WHEN "column" = 'weekday' THEN 'integer' 
                         WHEN "column" = 'hour' THEN 'integer'
                         WHEN "column" = 'minute' THEN 'integer'
                         END AS expected_datatype,
                    type AS actual_datatype,
                    CASE WHEN expected_datatype = actual_datatype THEN 'PASS'
                         WHEN expected_datatype != actual_datatype THEN 'FAIL'
                         END AS datatype_check_dates{i}
         FROM pg_table_def
         WHERE tablename = 'dates{i}' 
      """
    print(pd.read_sql_query(dc, conn))

    column expected_datatype actual_datatype datatype_check_dates2016
0  date_id           integer         integer                     PASS
1  case_id           integer         integer                     PASS
2     year           integer         integer                     PASS
3    month           integer         integer                     PASS
4      day           integer         integer                     PASS
5  weekday           integer         integer                     PASS
6     hour           integer         integer                     PASS
7   minute           integer         integer                     PASS
    column expected_datatype actual_datatype datatype_check_dates2017
0  date_id           integer         integer                     PASS
1  case_id           integer         integer                     PASS
2     year           integer         integer                     PASS
3    month           integer         integer                     PASS
4      day          

In [30]:
#location
for i in ['2016', '2017', '2018']:
    dc = f"""SELECT "column",
                    CASE WHEN "column" = 'loc_id' THEN 'integer' 
                         WHEN "column" = 'case_id' THEN 'integer' 
                         WHEN "column" = 'state' THEN 'character varying(256)' 
                         WHEN "column" = 'city' THEN 'character varying(256)'
                         WHEN "column" = 'county' THEN 'character varying(256)'
                         WHEN "column" = 'latitude' THEN 'numeric(18,0)' 
                         WHEN "column" = 'longitude' THEN 'numeric(18,0)'
                         END AS expected_datatype,
                    type AS actual_datatype,
                    CASE WHEN expected_datatype = actual_datatype THEN 'PASS'
                         WHEN expected_datatype != actual_datatype THEN 'FAIL'
                         END AS datatype_check_location{i}
         FROM pg_table_def
         WHERE tablename = 'location{i}' 
      """
    print(pd.read_sql_query(dc, conn))

      column       expected_datatype         actual_datatype  \
0     loc_id                 integer                 integer   
1    case_id                 integer                 integer   
2      state  character varying(256)  character varying(256)   
3       city  character varying(256)  character varying(256)   
4     county  character varying(256)  character varying(256)   
5   latitude           numeric(18,0)           numeric(18,0)   
6  longitude           numeric(18,0)           numeric(18,0)   

  datatype_check_location2016  
0                        PASS  
1                        PASS  
2                        PASS  
3                        PASS  
4                        PASS  
5                        PASS  
6                        PASS  
      column       expected_datatype         actual_datatype  \
0     loc_id                 integer                 integer   
1    case_id                 integer                 integer   
2      state  character varying(256)  c

In [31]:
#cars
for i in ['2016', '2017', '2018']:
    dc = f"""SELECT "column",
                    CASE WHEN "column" = 'car_id' THEN 'numeric(18,0)' 
                         WHEN "column" = 'case_id' THEN 'integer' 
                         WHEN "column" = 'make' THEN 'integer' 
                         WHEN "column" = 'model' THEN 'integer'
                         WHEN "column" = 'mod_year' THEN 'integer'
                         WHEN "column" = 'damage' THEN 'character varying(256)' 
                         WHEN "column" = 'dr_age' THEN 'integer'
                         WHEN "column" = 'dr_sex' THEN 'character varying(256)'
                         WHEN "column" = 'dr_injury' THEN 'character varying(256)'
                         WHEN "column" = 'dr_impair' THEN 'character varying(256)'
                         END AS expected_datatype,
                    type AS actual_datatype,
                    CASE WHEN expected_datatype = actual_datatype THEN 'PASS'
                         WHEN expected_datatype != actual_datatype THEN 'FAIL'
                         END AS datatype_check_cars{i}
         FROM pg_table_def
         WHERE tablename = 'cars{i}' 
      """
    print(pd.read_sql_query(dc, conn))

      column       expected_datatype         actual_datatype  \
0     car_id           numeric(18,0)           numeric(18,0)   
1    case_id                 integer                 integer   
2       make                 integer                 integer   
3      model                 integer                 integer   
4   mod_year                 integer                 integer   
5     damage  character varying(256)  character varying(256)   
6     dr_age                 integer                 integer   
7     dr_sex  character varying(256)  character varying(256)   
8  dr_injury  character varying(256)  character varying(256)   
9  dr_impair  character varying(256)  character varying(256)   

  datatype_check_cars2016  
0                    PASS  
1                    PASS  
2                    PASS  
3                    PASS  
4                    PASS  
5                    PASS  
6                    PASS  
7                    PASS  
8                    PASS  
9              

In [32]:
#non_motorists
for i in ['2016', '2017', '2018']:
    dc = f"""SELECT "column",
                    CASE WHEN "column" = 'nm_id' THEN 'numeric(18,0)' 
                         WHEN "column" = 'case_id' THEN 'integer' 
                         WHEN "column" = 'nm_type' THEN 'character varying(256)' 
                         WHEN "column" = 'nm_age' THEN 'integer'
                         WHEN "column" = 'nm_sex' THEN 'character varying(256)'
                         WHEN "column" = 'nm_injury' THEN 'character varying(256)' 
                         WHEN "column" = 'nm_impair' THEN 'character varying(256)'
                         END AS expected_datatype,
                    type AS actual_datatype,
                    CASE WHEN expected_datatype = actual_datatype THEN 'PASS'
                         WHEN expected_datatype != actual_datatype THEN 'FAIL'
                         END AS datatype_check_non_motorists{i}
         FROM pg_table_def
         WHERE tablename = 'non_motorists{i}' 
      """

    print(pd.read_sql_query(dc, conn))

      column       expected_datatype         actual_datatype  \
0      nm_id           numeric(18,0)           numeric(18,0)   
1    case_id                 integer                 integer   
2    nm_type  character varying(256)  character varying(256)   
3     nm_age                 integer                 integer   
4     nm_sex  character varying(256)  character varying(256)   
5  nm_injury  character varying(256)  character varying(256)   
6  nm_impair  character varying(256)  character varying(256)   

  datatype_check_non_motorists2016  
0                             PASS  
1                             PASS  
2                             PASS  
3                             PASS  
4                             PASS  
5                             PASS  
6                             PASS  
      column       expected_datatype         actual_datatype  \
0      nm_id           numeric(18,0)           numeric(18,0)   
1    case_id                 integer                 integer   

In [33]:
#crashes
for i in ['2016', '2017', '2018']:
    dc = f"""SELECT "column",
                    CASE WHEN "column" = 'case_id' THEN 'integer' 
                         WHEN "column" = 'date_id' THEN 'integer' 
                         WHEN "column" = 'loc_id' THEN 'integer'
                         WHEN "column" = 'car_id1' THEN 'numeric(18,0)'
                         WHEN "column" = 'car_id2' THEN 'numeric(18,0)' 
                         WHEN "column" = 'car_id3' THEN 'numeric(18,0)'
                         WHEN "column" = 'car_id4' THEN 'numeric(18,0)'
                         WHEN "column" = 'nm_id1' THEN 'numeric(18,0)'
                         WHEN "column" = 'nm_id2' THEN 'numeric(18,0)'
                         WHEN "column" = 'nm_id3' THEN 'numeric(18,0)'
                         WHEN "column" = 'nm_id4' THEN 'numeric(18,0)'
                         WHEN "column" = 'fatalities' THEN 'integer'
                         END AS expected_datatype,
                    type AS actual_datatype,
                    CASE WHEN expected_datatype = actual_datatype THEN 'PASS'
                         WHEN expected_datatype != actual_datatype THEN 'FAIL'
                         END AS datatype_check_crashes{i}
         FROM pg_table_def
         WHERE tablename = 'crashes{i}' 
      """
    print(pd.read_sql_query(dc, conn))

        column expected_datatype actual_datatype datatype_check_crashes2016
0      case_id           integer         integer                       PASS
1      date_id           integer         integer                       PASS
2       loc_id           integer         integer                       PASS
3      car_id1     numeric(18,0)   numeric(18,0)                       PASS
4      car_id2     numeric(18,0)   numeric(18,0)                       PASS
5      car_id3     numeric(18,0)   numeric(18,0)                       PASS
6      car_id4     numeric(18,0)   numeric(18,0)                       PASS
7       nm_id1     numeric(18,0)   numeric(18,0)                       PASS
8       nm_id2     numeric(18,0)   numeric(18,0)                       PASS
9       nm_id3     numeric(18,0)   numeric(18,0)                       PASS
10      nm_id4     numeric(18,0)   numeric(18,0)                       PASS
11  fatalities           integer         integer                       PASS
        colu

#### 4.3 Data dictionary 

**Fact Table**

1. **crashes**

    - case_id: accident case identifier - `st_case` column from the `accident` table
    - date_id: accident date identifier - autoincrement column from the `dates` table  
    - loc_id: accident location identifier - autoincrement column from the `location` table 
    - car_id1: car1/driver identifier - autoincrement column from the `cars` table    
    - car_id2: car2/driver identifier - autoincrement column from the `cars` table 
    - car_id3: car3/driver identifier - autoincrement column from the `cars` table 
    - car_id4: car4/driver identifier - autoincrement column from the `cars` table  
    - nm_id1: non-motorist1 identifier - autoincrement column from the `nmimpair` table   
    - nm_id2: non-motorist2 identifier - autoincrement column from the `nmimpair` table  
    - nm_id3: non-motorist3 identifier - autoincrement column from the `nmimpair` table  
    - nm_id4: non-motorist4 identifier - autoincrement column from the `nmimpair` table 
    - fatalities: number of deaths in the accident - `fatals` column from the `accident` table.  

**Dimension Tables**

2. **dates**
    - date_id: accident date identifier - autoincremented during `accident` table data extraction   
    - case_id: accident case identifier - `st_case` column from the `accident` table
    - year: year of the accident - `year` column from the `accident` table
    - month: month of the accident - `month` column from the `accident` table
    - day: day of the accident - `day` column from the `accident` table 
    - weekday: weekday of the accident - `day_week` column from the `accident` table 
    - hour: hour of the accident - `hour` column from the accident table 
    - minute: minute of the accident - `minute` column from the accident table  

3. **location**
    - loc_id: accident location identifier - autoincremented during `accident`/`glc` table data extraction  
    - case_id: accident case identifier - `st_case` column from the `accident` table
    - state: state of the accident location - `state_name` column from the `glc` table   
    - city: city of the accident location - `city_name` column from the `glc` table 
    - county: county of the accident location - `county_name` column from the `glc` table 
    - latitude: latitude coordinate of the accident location - `latitude` column from the `accident` table   
    - longitude: longitude coordinate of the accident location - `longitud` column from the `accident` table  
    
4. **cars**
    - car_id: car/driver identifier - autoincremented during `vehicle`/`person` table data extraction  
    - case_id: accident case identifier - `st_case` column from the `accident` table  
    - make: make of the vehicle - `make` column from the `vehicle` table  
    - model: model of the vehicle - `model` column from the `vehicle` table     
    - mod_year: model year of the vehicle - `mod_year` column from the `vehicle` table   
    - damage: damage level of the vehicle - `deformed` column from the `vehicle` table      
    - dr_age: vehicle driver age - `age` column from the `person` table  
    - dr_sex: vehicle driver sex - `sex` column from the `person` table 
    - dr_injury: vehicle driver injury level - `inj_sec` column from the `person` table   
    - dr_impair: vehicle driver impairment - `drimpair` column from the `drimpair` column
    
5. **non_motorists**
    - nm_id: non-motorist identifier - autoincremented during `person`/`nmimpair` table data extraction  
    - case_id: accident case identifier - `st_case` column from the `accident` table 
    - nm_type: non-motorst type - `per_typ` column from the `person` table   
    - nm_age: non-motorist age - `age` column from the `person` table 
    - nm_sex: non-motorist sex - `sex` column from the `person` table  
    - nm_injury: non-motorist injury level - `inj_sec` column from the `person` table    
    - nm_impair: non-motorist - `nmimpair` column from the `nmimpair` table

#### Step 5: Complete Project Write Up

##### Clearly state the rationale for the choice of tools and technologies for the project:
```
Redshift: Fast speed of copies and queries provided by MPP(Massively Parallel Processing) architecture, high compatibility and ease with S3 storage for data import and export. 

S3: High compatibility with Redshift, high capacity storage, ease of data transfer.

Postgres: Pivot functionality (crosstab function) from tablefunc module, same stack lineage as Redshift.
```

##### Propose how often the data should be updated and why:
```
If under study, the data should be updated annually - FARS publishes crash data annually.
```

##### Write a description of how you would approach the problem differently under the following scenarios:

##### The data was increased by 100x.**
```
1. Increase the number of nodes, processing power, and memory.

2. Designate distyle, distkeys, and sortkeys to optimize joins and reads.
```   
##### The data populates a dashboard that must be updated on a daily basis by 7am every day.**
```
ETL script will be set up on Airflow to daily schedule for 7AM.
```
##### The database needed to be accessed by 100+ people.
```
Ideally, all the requesters will be registered as AWS IAM users, but otherwise the requested tables will be exported and sent out as CSV/XLSX files.
```